In [20]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [28]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return len(entry)

In [22]:
def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [23]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

In [24]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 6))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 6))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [47]:
garbageCollectionDate[6]

'HobsonsBayCityCouncil-garbage-collection-zones.json'

In [25]:
readInJson(garbageCollectionDate[6]).head()

,type,name,features
0,FeatureCollection,Hbcc_Garbage_Collection_Zones,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
1,FeatureCollection,Hbcc_Garbage_Collection_Zones,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
2,FeatureCollection,Hbcc_Garbage_Collection_Zones,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
3,FeatureCollection,Hbcc_Garbage_Collection_Zones,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
4,FeatureCollection,Hbcc_Garbage_Collection_Zones,"{'type': 'Feature', 'geometry': {'type': 'Poly..."


In [36]:
df = readInJson(garbageCollectionDate[6])

df['properties'] = df.features.apply(lambda x: x['properties'])

df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0])

df = df.explode('geo')
df['geo_nested'] = df.geo.apply(lambda x: isNested(x))

nested = df.loc[df.geo_nested!=0].explode('geo')
nested['geo_nested'] = nested.geo.apply(lambda x: isNested(x))

df = pd.concat([df.loc[df.geo_nested==0], nested])

df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

df.drop(['geo', 'features', 'type', 'name', 'geo_nested'], axis = 1, inplace = True)

In [38]:
df.iloc[0,0]

{'Day': 'Tuesday',
 'Descr': 'Tuesday Area 2',
 'Cycle': '22T',
 'Land_Sch': 'R/2015-01-06/PW',
 'Recyc_Sch': 'R/2015-01-06/P2W',
 'Green_Sch': 'R/2015-01-13/P2W'}

In [41]:
newCols = ['landfill_colDay', 
           'recyclable_colDay',
          'green_colDay',]
newColsKeys = ['Land_Sch', 
               'Recyc_Sch', 
              'Green_Sch']

for key, val in dict(zip(newCols, newColsKeys)).items(): 
    df[key] = df.properties.apply(lambda x: x[val])

for i in ['landfill', 'recyclable', 'green']: 
    df[f'{i}_date'] = df[f'{i}_colDay'].apply(lambda x: x.split('/')[1])
    df[f'{i}_date'] = pd.to_datetime(df[f'{i}_date'], format='%Y-%m-%d').dt.day_name()
    df[f'{i}_freq'] = df[f'{i}_colDay'].apply(lambda x: x.split('/')[-1])
    df[f'{i}_freq'] = df[f'{i}_freq'].apply(lambda x: 'Every Second ' if x == 'P2W' else 'Every ')
    
    df[f'{i}_colDay'] = df[f'{i}_freq'] + df[f'{i}_date']
    
    df.drop([f'{i}_freq', f'{i}_date'], axis = 1, inplace = True)

df.drop('properties', axis = 1, inplace = True)

In [45]:
hobsonsBay = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay', 'green_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay', 'green_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'recyclable_colDay', 'green_colDay'], 
                           ['landfill', 'recyclable', 'organics'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

hobsonsBay.to_csv('../GBD_hobsonsBay.csv', index = False)